In [1]:
import pandas as pd

# Create sample data dictionary
data = {
    'date': ['2025-07-01', '2025-07-02', '2025-07-03'],
    'invoice_number': ['INV001', 'INV002', 'INV003'],
    'amount': [10000, 15000, 8000],
    'GSTIN': ['27AAAPL1234C1ZV', '', '27AAAPL1234C1ZV'],
    'tax_type': ['GST', 'GST', 'TDS']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Create a 'data' folder if not exists
import os
os.makedirs('data', exist_ok=True)

# Save to CSV file inside data folder
df.to_csv('data/sample_transactions.csv', index=False)

print("Sample CSV file created at data/sample_transactions.csv")


Sample CSV file created at data/sample_transactions.csv


In [2]:
import pandas as pd
import numpy as np
import os

# Prepare sample data for 10 entries
data = {
    'date': [
        '2025-07-01', '2025-07-02', '2025-07-03', '2025-07-04', '2025-07-05',
        '2025-07-06', '2025-07-07', '2025-07-08', '2025-07-09', '2025-07-10'
    ],
    'invoice_number': [
        'INV001', 'INV002', 'INV003', 'INV004', 'INV005',
        'INV006', 'INV007', 'INV008', 'INV009', 'INV010'
    ],
    'amount': [
        10000, 15000, 8000, 12000, 9000, 20000, 11000, 17000, 8500, 13500
    ],
    'GSTIN': [
        '27AAAPL1234C1ZV', '', '27AAAPL1234C1ZV', '29AABCU9603R1ZV', '',
        '24AAEPM5407B1ZF', '', '07AAACG2115R1Z2', '23AACCS2452K1ZT', ''
    ],
    'tax_type': [
        'GST', 'GST', 'TDS', 'GST', 'TDS', 'VAT', 'VAT', 'GST', 'TDS', 'VAT'
    ]
}

# Default tax rates
GST_RATE = 0.18
TDS_RATE = 0.10
VAT_RATE = 0.12

# Convert to DataFrame
df = pd.DataFrame(data)

# Assign rates as per tax_type
df['GST_rate'] = np.where(df['tax_type'] == 'GST', GST_RATE, 0.0)
df['TDS_rate'] = np.where(df['tax_type'] == 'TDS', TDS_RATE, 0.0)
df['VAT_rate'] = np.where(df['tax_type'] == 'VAT', VAT_RATE, 0.0)

# Compute amounts
df['GST_amount'] = df['amount'] * df['GST_rate']
df['TDS_amount'] = df['amount'] * df['TDS_rate']
df['VAT_amount'] = df['amount'] * df['VAT_rate']

# Compliance status
def check_compliance(row):
    if row['tax_type'] == 'GST' and not row['GSTIN']:
        return 'Missing GSTIN'
    if row['tax_type'] in ('GST', 'VAT') and not row['GSTIN']:
        return 'Missing GSTIN'
    return 'OK'

df['compliance_status'] = df.apply(check_compliance, axis=1)
df['remarks'] = ''

# Create data folder if not exists
os.makedirs('data', exist_ok=True)

# Save DataFrame to CSV
df.to_csv('data/sample_transactions.csv', index=False)

print('Sample CSV with all columns and 10 entries created at data/sample_transactions.csv')


Sample CSV with all columns and 10 entries created at data/sample_transactions.csv


In [8]:
import pandas as pd
import numpy as np
import json
import os

# Sample data as previously defined:
data = {
    'date': [
        '2025-07-01', '2025-07-02', '2025-07-03', '2025-07-04', '2025-07-05',
        '2025-07-06', '2025-07-07', '2025-07-08', '2025-07-09', '2025-07-10'
    ],
    'invoice_number': [
        'INV001', 'INV002', 'INV003', 'INV004', 'INV005',
        'INV006', 'INV007', 'INV008', 'INV009', 'INV010'
    ],
    'amount': [
        10000, 15000, 8000, 12000, 9000, 20000, 11000, 17000, 8500, 13500
    ],
    'GSTIN': [
        '27AAAPL1234C1ZV', '', '27AAAPL1234C1ZV', '29AABCU9603R1ZV', '',
        '24AAEPM5407B1ZF', '', '07AAACG2115R1Z2', '23AACCS2452K1ZT', ''
    ],
    'tax_type': [
        'GST', 'GST', 'TDS', 'GST', 'TDS', 'VAT', 'VAT', 'GST', 'TDS', 'VAT'
    ],
    # Optional: Include a GST category to use GST_RATES from rules file
    'GST_category': [
        'goods', 'services', 'goods', 'services', 'goods',
        'goods', 'services', 'goods', 'services', 'goods'
    ]
}

df = pd.DataFrame(data)

# Load tax rules from a JSON file (simulate here as a dict for demo)
tax_rules = {
    "GST_RATES": {
        "goods": 0.18,
        "services": 0.12
    },
    "TDS_RATES": {
        "default": 0.10,
        "special_slab": {
            "threshold": 10000,
            "rate_above_threshold": 0.15
        }
    },
    "VAT_RATES": {
        "default": 0.12
    }
}

# Define calculation functions using these rules:

def calculate_gst(amount, category):
    rate = tax_rules["GST_RATES"].get(category, 0)
    return amount * rate

def calculate_tds(amount):
    slab = tax_rules["TDS_RATES"]["special_slab"]
    if amount > slab["threshold"]:
        rate = slab["rate_above_threshold"]
    else:
        rate = tax_rules["TDS_RATES"]["default"]
    return amount * rate

def calculate_vat(amount):
    rate = tax_rules["VAT_RATES"]["default"]
    return amount * rate

# Apply calculations based on tax type & category
df['GST_amount'] = df.apply(lambda row: calculate_gst(row['amount'], row.get('GST_category', 'goods')) if row['tax_type'] == 'GST' else 0, axis=1)
df['TDS_amount'] = df.apply(lambda row: calculate_tds(row['amount']) if row['tax_type'] == 'TDS' else 0, axis=1)
df['VAT_amount'] = df.apply(lambda row: calculate_vat(row['amount']) if row['tax_type'] == 'VAT' else 0, axis=1)

# Compliance checker function that flags issues
def check_compliance(df, rules):
    errors = []
    for idx, row in df.iterrows():
        # Check GSTIN presence for GST and VAT
        if row['tax_type'] in ['GST', 'VAT'] and not row['GSTIN']:
            errors.append(f"Row {idx}: Missing GSTIN for tax_type {row['tax_type']}")

        # Verify GST amount calculation roughly matches expected given GST_rate
        if row['tax_type'] == 'GST':
            expected_gst = calculate_gst(row['amount'], row.get('GST_category', 'goods'))
            if not np.isclose(row['GST_amount'], expected_gst, atol=1e-2):
                errors.append(f"Row {idx}: GST amount mismatch")

        # Check if amount exceeds TDS slab threshold but TDS rate not applied correctly
        if row['tax_type'] == 'TDS':
            expected_tds = calculate_tds(row['amount'])
            if not np.isclose(row['TDS_amount'], expected_tds, atol=1e-2):
                errors.append(f"Row {idx}: TDS amount mismatch")

        # Add more rules as needed...

    return errors

# Run compliance check
compliance_errors = check_compliance(df, tax_rules)

# Add compliance status column summarizing presence of errors per row
df['compliance_status'] = 'OK'
for err in compliance_errors:
    # Extract row index from error message, e.g., "Row 2: ..."
    idx = int(err.split(':')[0].split()[1])
    df.loc[idx, 'compliance_status'] = 'Error'

# Add remarks combining any errors per row
df['remarks'] = ''
for err in compliance_errors:
    idx = int(err.split(':')[0].split()[1])
    df.loc[idx, 'remarks'] += err + "; "

# Create 'data' folder and save updated file
os.makedirs('data', exist_ok=True)
df.to_csv('data/sample_transactions.csv', index=False)

print("Processed transactions with tax calculations and compliance checks saved to data/sample_transactions_with_checks.csv")

# Optionally print errors
for e in compliance_errors:
    print("Compliance issue:", e)



Processed transactions with tax calculations and compliance checks saved to data/sample_transactions_with_checks.csv
Compliance issue: Row 1: Missing GSTIN for tax_type GST
Compliance issue: Row 6: Missing GSTIN for tax_type VAT
Compliance issue: Row 9: Missing GSTIN for tax_type VAT


In [10]:
!pip install openpyxl



   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [12]:
!C:\Users\admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 2.2 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 2.5 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [11]:
import pandas as pd
import numpy as np
import json
import os

# (Assuming your existing data & code here up to compliance checks...)

# Sample data and tax rules as given previously (not repeating here for brevity)

# -- Existing calculations and compliance checks --

# After compliance checks and DataFrame updates, generate draft reports:

# Create output folder
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)

# Generate GST return report (e.g., sum per GSTIN and category)
gst_df = df[df['tax_type'] == 'GST'].copy()
# Summarize GST by GSTIN and GST_category, summing taxable amount and GST_amount
gst_summary = gst_df.groupby(['GSTIN', 'GST_category']).agg(
    total_taxable_amount=pd.NamedAgg(column='amount', aggfunc='sum'),
    total_gst_amount=pd.NamedAgg(column='GST_amount', aggfunc='sum')
).reset_index()
gst_summary.to_csv(os.path.join(output_folder, 'gst_return_report.csv'), index=False)
gst_summary.to_excel(os.path.join(output_folder, 'gst_return_report.xlsx'), index=False)

# Generate TDS summary report (sum per GSTIN if applicable or general summary)
tds_df = df[df['tax_type'] == 'TDS'].copy()
tds_summary = tds_df.groupby('GSTIN').agg(
    total_tdsable_amount=pd.NamedAgg(column='amount', aggfunc='sum'),
    total_tds_amount=pd.NamedAgg(column='TDS_amount', aggfunc='sum')
).reset_index()
tds_summary.to_csv(os.path.join(output_folder, 'tds_summary_report.csv'), index=False)
tds_summary.to_excel(os.path.join(output_folder, 'tds_summary_report.xlsx'), index=False)

# Generate VAT return report (sum per GSTIN if applicable or general summary)
vat_df = df[df['tax_type'] == 'VAT'].copy()
vat_summary = vat_df.groupby('GSTIN').agg(
    total_vatable_amount=pd.NamedAgg(column='amount', aggfunc='sum'),
    total_vat_amount=pd.NamedAgg(column='VAT_amount', aggfunc='sum')
).reset_index()
vat_summary.to_csv(os.path.join(output_folder, 'vat_return_report.csv'), index=False)
vat_summary.to_excel(os.path.join(output_folder, 'vat_return_report.xlsx'), index=False)

print(f"Draft GST, TDS, VAT reports generated in folder: {output_folder}")


Draft GST, TDS, VAT reports generated in folder: output
